In [1]:
from __future__ import print_function
from six.moves import cPickle as pickle
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
import tensorflow as tf

In [2]:
f=open('Data.pickle','rb')
Data=open('Data.pickle','rb')
train_dataset=pickle.load(Data)['train_dataset']
Data.seek(0)
train_labels=pickle.load(Data)['train_labels']
Data.seek(0)
test_dataset=pickle.load(Data)['test_dataset']
Data.seek(0)
test_labels=pickle.load(Data)['test_labels']
Data.close()

In [3]:
count=0
Corrupted_data=np.zeros((28,28),dtype=np.float32)
for i in range(train_dataset.shape[0]):
    if np.array_equal(train_dataset[i,:,:],Corrupted_data):
        Train_data[count,:,:]=train_dataset[i,:,:]
        Train_label[count]=train_labels[i]
        count=count+1
print(count)        
print(train_labels.shape)
print(train_dataset.shape)

train_dataset=train_dataset.reshape([train_dataset.shape[0],train_dataset.shape[1],train_dataset.shape[2],1])
test_dataset=test_dataset.reshape([test_dataset.shape[0],test_dataset.shape[1],test_dataset.shape[2],1])
print(test_dataset.shape)
print(train_dataset.shape)
print(test_labels.shape)

0
(50000,)
(50000, 28, 28)
(900, 28, 28, 1)
(50000, 28, 28, 1)
(900,)


In [4]:
def One_hot_encoder(labels,C):
    C=tf.constant(C,dtype=tf.int32,name='C')
    encoded=tf.one_hot(labels,C)
    with tf.Session() as sess:
        return sess.run(encoded)

In [5]:
Train_labels=One_hot_encoder(train_labels,10)
Test_labels=One_hot_encoder(test_labels,10)
print(Train_labels.shape)

(50000, 10)


In [6]:
def create_placeholders(n_H0, n_W0, n_C0, n_y):
    X = tf.placeholder(shape=(None, n_H0, n_W0, n_C0),dtype=tf.float32)
    Y = tf.placeholder(shape=(None, n_y),dtype=tf.float32)
    return X, Y

In [7]:
def initialize_parameters():
    W1 = tf.get_variable('W1',[4,4,1,8],initializer=tf.contrib.layers.xavier_initializer(seed=0))
    W2 = tf.get_variable('W2',[2,2,8,16],initializer=tf.contrib.layers.xavier_initializer(seed=0))
    parameters = {"W1": W1,
                  "W2": W2}
    return parameters

In [8]:
def forward_propagation(X, parameters):
    W1 = parameters['W1']
    W2 = parameters['W2']
    Z1 = tf.nn.conv2d(X,W1,strides=[1,1,1,1],padding='SAME')
    A1 = tf.nn.relu(Z1)
    P1 = tf.nn.max_pool(A1,padding='SAME',ksize=[1,8,8,1],strides=[1,8,8,1])
    Z2 = tf.nn.conv2d(P1,W2,strides=[1,1,1,1],padding='SAME')
    A2 = tf.nn.relu(Z2)
    P2 = tf.nn.max_pool(A2,padding='SAME',ksize=[1,4,4,1],strides=[1,4,4,1])
    P2 = tf.contrib.layers.flatten(P2)
    Z3 = tf.contrib.layers.fully_connected(P2,10,activation_fn=None)
    return Z3

In [9]:
def compute_cost(Z3, Y):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = Z3, labels = Y))
    return cost

In [10]:
import time as t
X, Y = create_placeholders(28,28,1,10)
parameters = initialize_parameters()
Z3 = forward_propagation(X, parameters)
cost = compute_cost(Z3, Y)
optimizer = tf.train.AdamOptimizer().minimize(cost)
init = tf.global_variables_initializer()
predict_op = tf.argmax(Z3, 1)
correct_prediction = tf.equal(predict_op, tf.argmax(Y, 1))        
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
with tf.Session() as sess:
    sess.run(init)
    print("Training Started")
    A=t.time()
    for i in range(1500):
        _ , temp_cost,acc = sess.run([optimizer,cost,accuracy],{X:train_dataset,Y:Train_labels})
        print(i," : ",temp_cost," : ",acc)
    print('Training Over time taken ',t.time()-A,'sec')
    A=t.time()
    print('Testing Accuracy',sess.run(accuracy,{X:test_dataset,Y:Test_labels}))
    print('Testing Time',t.time()-A)
    

Training Started
0  :  2.40939  :  0.09244
1  :  2.39925  :  0.0902
2  :  2.38984  :  0.08804
3  :  2.38116  :  0.08674
4  :  2.37318  :  0.086
5  :  2.36585  :  0.0864
6  :  2.35916  :  0.08578
7  :  2.35303  :  0.08698
8  :  2.34743  :  0.08894
9  :  2.34231  :  0.0921
10  :  2.33763  :  0.09632
11  :  2.33333  :  0.09918
12  :  2.32937  :  0.1033
13  :  2.32571  :  0.10588
14  :  2.3223  :  0.1095
15  :  2.31913  :  0.11224
16  :  2.31615  :  0.11398
17  :  2.31334  :  0.11496
18  :  2.31068  :  0.11492
19  :  2.30815  :  0.11594
20  :  2.30572  :  0.1166
21  :  2.30337  :  0.11698
22  :  2.30108  :  0.11718
23  :  2.29885  :  0.11692
24  :  2.29665  :  0.11688
25  :  2.29448  :  0.11662
26  :  2.29234  :  0.11756
27  :  2.2902  :  0.11808
28  :  2.28808  :  0.11986
29  :  2.28596  :  0.12216
30  :  2.28385  :  0.12486
31  :  2.28178  :  0.12906
32  :  2.27974  :  0.13306
33  :  2.27773  :  0.13704
34  :  2.27573  :  0.14206
35  :  2.27372  :  0.14704
36  :  2.27172  :  0.152
37  : 

In [1]:
import urllib3, requests, json
service_path = "https://ibm-watson-ml.mybluemix.net"
username = "fa1cc9c9-706e-43ba-8c6a-6a7263bb469d"
password = "940a6a32-4fa5-4ada-b991-853169740e5d"
headers = urllib3.util.make_headers(basic_auth='{}:{}'.format(username, password))
url = '{}/v3/identity/token'.format(service_path)
response = requests.get(url, headers=headers)
ml_token = 'Bearer ' + json.loads(response.text).get('token')

In [2]:
instance_id = "0d289a1b-6394-4dfa-ace7-be894e3d03d7"
endpoint_instance = service_path + "/v3/wml_instances/" + instance_id
header = {'Content-Type': 'application/json', 'Authorization': ml_token}

response_get_instance = requests.get(endpoint_instance, headers=header)
print(response_get_instance)
print(response_get_instance.text)

<Response [200]>
{
  "metadata": {
    "guid": "0d289a1b-6394-4dfa-ace7-be894e3d03d7",
    "url": "https://ibm-watson-ml.mybluemix.net/v3/wml_instances/0d289a1b-6394-4dfa-ace7-be894e3d03d7",
    "created_at": "2018-04-10T06:46:42.383Z",
    "modified_at": "2018-04-10T06:46:42.383Z"
  },
  "entity": {
    "source": "Bluemix",
    "published_models": {
      "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/0d289a1b-6394-4dfa-ace7-be894e3d03d7/published_models"
    },
    "usage": {
      "expiration_date": "2018-08-01T00:00:00.000Z",
      "computation_time": {
        "limit": 180000,
        "current": 1779
      },
      "model_count": {
        "limit": 200,
        "current": 2
      },
      "prediction_count": {
        "limit": 5000,
        "current": 4
      },
      "gpu_count": {
        "limit": 8,
        "current": 0
      },
      "capacity_units": {
        "limit": 180000000,
        "current": 1779214
      },
      "deployment_count": {
        "limit": 5,
 

In [3]:
endpoint_published_models = json.loads(response_get_instance.text).get('entity').get('published_models').get('url')
print(endpoint_published_models)
header = {'Content-Type': 'application/json', 'Authorization': ml_token}
response_get = requests.get(endpoint_published_models, headers=header)
print(response_get)
print(response_get.text)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/0d289a1b-6394-4dfa-ace7-be894e3d03d7/published_models
<Response [200]>
{
  "limit": 1000,
  "resources": [{
    "metadata": {
      "guid": "551dd972-ec1f-4fa0-8940-766251bf00db",
      "url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/0d289a1b-6394-4dfa-ace7-be894e3d03d7/published_models/551dd972-ec1f-4fa0-8940-766251bf00db",
      "created_at": "2018-07-15T16:44:05.191Z",
      "modified_at": "2018-07-15T16:46:18.633Z"
    },
    "entity": {
      "runtime_environment": "spark-2.1",
      "learning_configuration_url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/0d289a1b-6394-4dfa-ace7-be894e3d03d7/published_models/551dd972-ec1f-4fa0-8940-766251bf00db/learning_configuration",
      "name": "House Prices: Advanced Regression Techniques",
      "label_col": "SalePrice",
      "learning_iterations_url": "https://us-south.ml.cloud.ibm.com/v3/wml_instances/0d289a1b-6394-4dfa-ace7-be894e3d03d7/published_models/551dd972-ec1f

In [ ]:
[endpoint_deployments] = [x.get('entity').get('deployments').get('url') 
for x in json.loads(response_get.text).get('resources') if x.get('metadata').get('guid') == saved_model.uid]
print(endpoint_deployments)

In [ ]:
payload_online = {"name": "Product Line Prediction", "description": "My Cool Deployment", "type": "online"}
response_online = requests.post(endpoint_deployments, json=payload_online, headers=header)

print(response_online)
print(response_online.text)

In [ ]:
scoring_url = json.loads(response_online.text).get('entity').get('scoring_url')
print(scoring_url)